In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

In [2]:
Base = declarative_base()

C:\Users\jfand\AppData\Local\Temp\ipykernel_18596\4196137762.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
#Criar duas classes representando tabelas no banco de dados

class Usuario(Base):
    __tablename__ = 'usuarios'
    id = Column(Integer, primary_key=True)
    nome = Column(String)
    idade = Column(Integer)
    pedidos = relationship('Pedido', back_populates='usuario')

class Pedido(Base):
    __tablename__ = 'pedidos'
    id = Column(Integer, primary_key=True)
    descricao = Column(String)
    usuario_id = Column(Integer, ForeignKey('usuarios.id'))
    usuario = relationship('Usuario', back_populates='pedidos')

In [4]:
#Criar um banco de dados SQLite em memória

engine = create_engine('sqlite:///:memory:', echo=False)
Base.metadata.create_all(engine)

#Criar uma sessão para interagir com o banco de dados

Session = sessionmaker(bind=engine)
session = Session()

In [5]:
#Inserir usuarios e pedidos no banco de dados

usuario1 = Usuario(nome='João', idade=25)
usuario2 = Usuario(nome='Maria', idade=22)

pedido1 = Pedido(descricao='Teclado', usuario=usuario1)
pedido2 = Pedido(descricao='Mouse', usuario=usuario1)
pedido3 = Pedido(descricao='Monitor', usuario=usuario2)

session.add_all([usuario1, usuario2, pedido1, pedido2, pedido3])
session.commit()

In [8]:
#Consultar e exibir todos os usuários e pedidos

usuarios = session.query(Usuario).all()
for usuario in usuarios:
    print(f'Usuário: {usuario.nome}, {usuario.idade} anos')
    #Cada usuário pode ter vários pedidos
    for pedido in usuario.pedidos:
        print(f'  Pedido: {pedido.descricao}')
    print('\n')


Usuário: João, 25 anos
  Pedido: Teclado
  Pedido: Mouse


Usuário: Maria, 22 anos
  Pedido: Monitor




In [9]:
#Atualizar a idade de um usuário

usuario = session.query(Usuario).filter_by(nome='João').first()
usuario.idade = 26
session.commit()



In [10]:
#Excluir um usuário e seus pedidos associados

usuario = session.query(Usuario).filter_by(nome='Maria').first()
session.delete(usuario)

session.commit()


In [13]:
#Buscar pedidos contendo uma palavra-chave

pedidos = session.query(Pedido).filter(Pedido.descricao.like('%Teclado%')).all()
for pedido in pedidos:
    print(f'Pedido: {pedido.descricao} do usuário {pedido.usuario.nome} com o id {pedido.usuario_id}')

Pedido: Teclado do usuário João com o id 1
